# AIR - Exercise in Google Colab

## Colab Preparation

Open via google drive -> right click: open with Colab

**Get a GPU**

Toolbar -> Runtime -> Change Runtime Type -> GPU

**Mount Google Drive**

* Download data and clone your github repo to your Google Drive folder
* Use Google Drive as connection between Github and Colab (Could also use direct github access, but re-submitting credentials might be annoying)
* Commit to Github locally from the synced drive

**Keep Alive**

When training google colab tends to kick you out, This might help: https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0

**Get Started**

Run the following script to mount google drive and install needed python packages. Pytorch comes pre-installed.

In [19]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

!pip install transformers

path="/content/drive/MyDrive/AIR/AIR/air-exercise-2/Part-3/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
qa_tuples = pd.read_csv(r'{}msmarco-fira-21.qrels.qa-tuples.tsv'.format(path),sep='8921349812748}',header=None,names=['entry'])
qa_tuples.entry = qa_tuples.entry.str.split('\t')
def fix_tuples(row):
    return [row[0],row[1],row[2],row[3],row[4],row[6:]]

qa_tuples.entry = (qa_tuples.entry.apply(fix_tuples))
qa_tuples = pd.DataFrame(qa_tuples["entry"].to_list(), columns=['queryid', 'documentid', 'relevance_grade', 'query_text', 'document_text', 'text_selection'])

def str_start(row):
    start_index = []
    for string in row.text_selection:
        start_index.append(row.document_text.index(string))
    return {'answer_start':start_index,'text':row.text_selection}

qa_tuples['answers'] = qa_tuples.apply(str_start,axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [21]:
from transformers import AutoModelForQuestionAnswering,  AutoTokenizer, pipeline

model_name = "deepset/minilm-uncased-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

print(res)

{'score': 0.33747559785842896, 'start': 65, 'end': 84, 'answer': 'freedom to the user'}


In [22]:
def get_answers(row):
  question = row.query_text
  context = row.document_text

  QA_input = {
    'question': question,
    'context': context
  } 

  return nlp(QA_input)['answer']


In [23]:
qa_tuples['model_answer'] = qa_tuples.apply(get_answers,axis=1)

In [24]:
from core_metrics import normalize_answer,get_tokens,compute_exact,compute_f1

In [25]:
def calculate_answer_scores(data):

  real_answers = data['text_selection']
  model_answer = data['model_answer']


  f1_scores = []
  exact_scores = []
  contained = []

  for answer in real_answers:
    f1_scores.append(compute_f1(answer,model_answer))
    exact_scores.append(compute_exact(answer,model_answer))
    
    if answer in model_answer:
      contained.append(1)
    else: 
      contained.append(0)
      

  return max(f1_scores),max(exact_scores),max(contained)

In [26]:
qa_tuples[['F1_score','Exact_score',"Contained"]] = qa_tuples.apply(calculate_answer_scores,axis=1, result_type='expand')

In [30]:
qa_tuples

,queryid,documentid,relevance_grade,query_text,document_text,text_selection,answers,model_answer,F1_score,Exact_score,Contained
0,135386,100163,3,definition of imagination,imagination - the formation of a mental image ...,[the formation of a mental image of something ...,"{'answer_start': [14], 'text': ['the formation...",imagination,0.000000,0.0,0.0
1,290779,101026,3,how many oscars has clint eastwood won?pdrijgh...,Clint Eastwood -- five-time Oscar winner and e...,[five],"{'answer_start': [18], 'text': ['five']}",five,1.000000,1.0,1.0
2,21741,1021598,3,are cold sores and fever blisters the same,"Cold sores, sometimes called fever blisters, a...","[Cold sores, sometimes called fever blisters]","{'answer_start': [0], 'text': ['Cold sores, so...","The skin around the blisters is often red, swo...",0.133333,0.0,0.0
3,810210,1029662,3,what is the cause of blood in the stool,Having blood in the stool can be the result of...,"[wide variety of conditions, such as hemorrhoi...","{'answer_start': [49], 'text': ['wide variety ...",Having blood in the stool can be the result of...,0.205128,0.0,0.0
4,1097448,103635,3,how many calories in slim fast shakes,"The chocolate-flavored shake contains 190, whi...","[chocolate-flavored shake contains 190, Cafe C...","{'answer_start': [4, 53], 'text': ['chocolate-...",each provide about 10 percent of the calories,0.142857,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
52601,525779,4877404,2,twin tower adress,The twin towers were built in the borough of M...,[The twin towers were built in the borough of ...,"{'answer_start': [0, 55, 167], 'text': ['The t...",he Twin Towers are not being rebuilt,0.266667,0.0,0.0
52602,210442,4877731,3,how can i get more energy while pregnant,To compensate for this your body will require ...,[To compensate for this your body will require...,"{'answer_start': [0], 'text': ['To compensate ...",your body will require an increase in energy o...,0.377358,0.0,0.0
52603,1088928,4878423,3,"vitamins a, d, e, and k are dependent upon","Four important fat-soluble vitamins are A, D, ...","[Vitamins A, D, and K cooperate synergisticall...","{'answer_start': [56, 178], 'text': ['Vitamins...",cooperate synergistically,0.200000,0.0,0.0
52604,550565,4881591,3,what age can you wear baby on back in a carrier?,When can I carry my baby in a front pack facin...,[As soon as your baby can hold his head up ste...,"{'answer_start': [53], 'text': ['As soon as yo...",4 to 6 months,0.173913,0.0,0.0


In [27]:
def summarize_results(data):

  print(data.Exact_score.sum())
  print(data.Contained.sum())
  print(data.F1_score.sum()/data.shape[0])

In [28]:
summarize_results(qa_tuples)

6867.0
9441.0
0.43677574759651416


In [31]:
qa_tuples.to_csv('{}Model_answers.csv'.format(path))

In [32]:
qa_tuples.shape[0]

52606